## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS", "STATUS", "ASK_AMT"], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1


In [4]:
# Determine the number of unique values in each column.
nununique_column = application_df.nunique()
print(nununique_column)

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_types_count = application_df['APPLICATION_TYPE'].value_counts()
print(application_types_count)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# define the cuttoff value
cutoff_value = 500

# Filter the application types with counts below the cutoff value
application_types_below_cutoff = application_types_count[application_types_count < cutoff_value]

# getting the indexs of applications types whcih are below to the cutoff value
application_types_to_replace = application_types_below_cutoff.index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
print(classification_count)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_above_1 = classification_count[classification_count > 1]
print(classifications_above_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# define the cuttoff value
cutoff_value = 1000

# Filter the application types with counts below the cutoff value
classifications_below_cutoff = classification_count[classification_count < cutoff_value]

# getting the indexs of applications types whcih are below to the cutoff value
classifications_to_replace = classifications_below_cutoff.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded = application_df_encoded.astype(int)

print(application_df_encoded.head())

   IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0              1                       0                     1   
1              1                       0                     0   
2              0                       0                     0   
3              1                       0                     0   
4              1                       0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     0                    0                    0   
1                     0                    1                    0   
2                     0                    0                    0   
3                     0                    1                    0   
4                     0                    1                    0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  \
0                    0                    0                    0   
1                    0                    0         

In [11]:
# Split our preprocessed data into our features and target arrays
Y = application_df_encoded['IS_SUCCESSFUL'].values
X = application_df_encoded.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# defining hidden units
hidden_units_layer1 = 12
hidden_units_layer2 = 6
hidden_units_layer3 = 1

# define number of input features'
num_input_features = len(X_train_scaled[0])

# creating the sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer1, input_dim = num_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer3, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                480       
                                                                 
 dense_1 (Dense)             (None, 6)                 78        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 565 (2.21 KB)
Trainable params: 565 (2.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# # Define the checkpoint path and filenames
# os.makedirs("/content/drive/MyDrive/Challenge 21/Epochs/",exist_ok=True)
# checkpoint_path = "/content/drive/MyDrive/Challenge 21/Epochs/weights.{epoch:02d}.hdf5"

In [18]:
# # Create a callback that saves the model's weights every 5 epochs
# callbacks = ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=1,
#     save_weights_only=True,
#     save_freq=4000)

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
model_trained = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 5s 3ms/step - loss: 0.6063 - accuracy: 0.6809
Epoch 2/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5636 - accuracy: 0.7235
Epoch 3/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5584 - accuracy: 0.7262
Epoch 4/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5558 - accuracy: 0.7282
Epoch 5/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5546 - accuracy: 0.7283
Epoch 6/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5536 - accuracy: 0.7298
Epoch 7/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7295
Epoch 8/100
858/858 [==============================] - 4s 4ms/step - loss: 0.5517 - accuracy: 0.7298
Epoch 9/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5508 - accuracy: 0.7299
Epoch 10/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5505 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5488 - accuracy: 0.7340 - 475ms/epoch - 2ms/step
Loss: 0.5487977862358093, Accuracy: 0.7339650392532349


In [17]:
# Export our model to HDF5 file
nn.save('/content/drive/MyDrive/Challenge 21/AphabetSoup_Charity_Prediction_model_optimize_v1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Optimization model 02 by adding some hidden layers

In [18]:
# defining hidden units
hidden_units_layer1 = 24
hidden_units_layer2 = 12
hidden_units_layer3 = 6
hidden_units_layer4 = 1

# define number of input features'
num_input_features = len(X_train_scaled[0])

# creating the sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer1, input_dim = num_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=hidden_units_layer4, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                960       
                                                                 
 dense_4 (Dense)             (None, 12)                300       
                                                                 
 dense_5 (Dense)             (None, 6)                 78        
                                                                 
 dense_6 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1345 (5.25 KB)
Trainable params: 1345 (5.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# defining sgd optimizer
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=sgd_optimizer, metrics=["accuracy"])

In [21]:
# Train the model
model_trained = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 4s 3ms/step - loss: 0.5820 - accuracy: 0.7042
Epoch 2/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5585 - accuracy: 0.7267
Epoch 3/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5549 - accuracy: 0.7278
Epoch 4/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5533 - accuracy: 0.7286
Epoch 5/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5525 - accuracy: 0.7287
Epoch 6/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5518 - accuracy: 0.7292
Epoch 7/100
858/858 [==============================] - 4s 4ms/step - loss: 0.5512 - accuracy: 0.7307
Epoch 8/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5508 - accuracy: 0.7304
Epoch 9/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5500 - accuracy: 0.7301
Epoch 10/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5492 - accura

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5507 - accuracy: 0.7348 - 552ms/epoch - 3ms/step
Loss: 0.550700306892395, Accuracy: 0.734839677810669


In [24]:
# Export our model to HDF5 file
nn.save('/content/drive/MyDrive/Challenge 21/AphabetSoup_Charity_Prediction_model_optimize_v2.h5')

Optimizing the model verion 03

In [3]:
# loading the dataset
application_df_1 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")

application_df_1.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# drop non benficial columns
application_df_1.drop(columns=["EIN", "NAME"], inplace=True)
application_df_1.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
nununique_column = application_df_1.nunique()
print(nununique_column)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_types_count = application_df_1['APPLICATION_TYPE'].value_counts()
print(application_types_count)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# define the cuttoff value
cutoff_value = 500

# Filter the application types with counts below the cutoff value
application_types_below_cutoff = application_types_count[application_types_count < cutoff_value]

# getting the indexs of applications types whcih are below to the cutoff value
application_types_to_replace = application_types_below_cutoff.index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_1['APPLICATION_TYPE'] = application_df_1['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_1['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df_1['CLASSIFICATION'].value_counts()
print(classification_count)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_above_1 = classification_count[classification_count > 1]
print(classifications_above_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# define the cuttoff value
cutoff_value = 1000

# Filter the application types with counts below the cutoff value
classifications_below_cutoff = classification_count[classification_count < cutoff_value]

# getting the indexs of applications types whcih are below to the cutoff value
classifications_to_replace = classifications_below_cutoff.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_1['CLASSIFICATION'] = application_df_1['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df_1['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df_1)
application_df_encoded = application_df_encoded.astype(int)

print(application_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  

In [12]:
# Split our preprocessed data into our features and target arrays
Y = application_df_encoded['IS_SUCCESSFUL'].values
X = application_df_encoded.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
input_features = len(X_train_scaled[0])

In [31]:
def create_model(hp):
  nn_model = tf.keras.models.Sequential()

  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                                  min_value=1,
                                                  max_value=90,
                                                  step=5), activation=activation, input_dim=input_features))

  for i in range(hp.Int('num_layers', 1, 5)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_'+str(i),
                                                  min_value=1,
                                                  max_value=90,
                                                  step=5), activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return nn_model

In [16]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [32]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2
)

In [33]:
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 42s]
val_accuracy: 0.7368804812431335

Best val_accuracy So Far: 0.7403790354728699
Total elapsed time: 00h 26m 03s

Search: Running Trial #61

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
56                |86                |first_units
4                 |4                 |num_layers
6                 |86                |units_0
51                |56                |units_1
6                 |71                |units_2
6                 |16                |units_3
71                |46                |units_4
6                 |6                 |tuner/epochs
0                 |2                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/6
858/858 [==============================] - 6s 5ms/step - loss: 0.5731 - accuracy: 0.7200 - val_loss: 0.5555 - val_accuracy: 0.7329
Epoch 2/6
858/858 [==========================

KeyboardInterrupt: 